<a href="https://colab.research.google.com/github/CD-GRUPO3/TA047R-1C2025-GRUPO03/blob/main/TA047R_TP1_GRUPO03_ENTREGA_EJ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import pyarrow.parquet as pq


In [3]:
!gdown 1MukhnrymZ35FVsahXUs5I0cAeGJMoXgG
!gdown 1hEhbxXEC7Dc4iG0fKk2k1n8KM6emWxSg
!gdown 1iju9XQghLYI4-XcDt1TVDE1O9ZwRyFVg

Downloading...
From: https://drive.google.com/uc?id=1MukhnrymZ35FVsahXUs5I0cAeGJMoXgG
To: /content/yellow_tripdata_2024-08.parquet
100% 51.1M/51.1M [00:00<00:00, 66.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hEhbxXEC7Dc4iG0fKk2k1n8KM6emWxSg
To: /content/yellow_tripdata_2024-09.parquet
100% 61.2M/61.2M [00:01<00:00, 57.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iju9XQghLYI4-XcDt1TVDE1O9ZwRyFVg
To: /content/yellow_tripdata_2024-07.parquet
100% 52.3M/52.3M [00:00<00:00, 188MB/s]


In [4]:
trips1 = pq.read_table('/content/yellow_tripdata_2024-07.parquet')
data1 = trips1.to_pandas()

trips2 = pq.read_table('/content/yellow_tripdata_2024-08.parquet')
data2 = trips2.to_pandas()

trips3 = pq.read_table('/content/yellow_tripdata_2024-09.parquet')
data3 = trips3.to_pandas()


#Unimos los datos de cada uno de los 3 meses
data= pd.concat([data1,data2,data3],axis=0, ignore_index=True)

In [21]:
#VendorID                 Cualitativa    numerica   nominal
#tpep_pickup_datetime     Cuantitativa   discreta
#tpep_dropoff_datetime    Cuantitativa   discreta
#passenger_count          Cuantitativa    numerica   nominal
#trip_distance            Cuantitativa    numerica   nominal
#RatecodeID               Cualitativa    numerica   ordinal
#store_and_fwd_flag       Cualitativa    texto      nominal
#PULocationID             Cualitativa    numerica   nominal
#DOLocationID             Cualitativa    numerica   nominal
#payment_type             Cualitativa    numerica   nominal
#fare_amount              Cuantitativa   discreta    (???!!!)
#extra                    Cuantitativa   discreta    (???!!!)
#mta_tax                  Cuantitativa   discreta    (?)
#tip_amount               Cuantitativa   discreta    (???!!!)
#tolls_amount             Cuantitativa   discreta    (???!!!)
#improvement_surcharge    Cuantitativa   discreta    (?)
#total_amount             Cuantitativa   discreta    (???!!!)
#congestion_surcharge     Cuantitativa   discreta    (?)
#Airport_fee              Cuantitativa   discreta    (?)

In [20]:
lista_cuantitativas = ['passenger_count','trip_distance','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','Airport_fee'] #tpep_dropoff_datetime   y   tpep_pickup_datetime
lista_cualitativas = ['VendorID','RatecodeID','store_and_fwd_flag','PULocationID','DOLocationID','payment_type']

for columna in lista_cuantitativas:

  print(data[columna].describe())

  print('\n')

  '''
  promedio = data[columna].mean()   #No tomamos los NaN!!!
  print("El promedio de "+ columna+ " es:", promedio)


  mediana =  data[columna].median() #No tomamos los NaN!!!
  print("La mediana de "+ columna+ " es:", mediana)

  moda = data[columna].mode()[0]
  print("La moda de "+ columna+ " es:", moda)

  '''





count    8.667729e+06
mean     1.337451e+00
std      8.119679e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.000000e+00
Name: passenger_count, dtype: float64
La mediana de passenger_count es: 1.0


count    9.689116e+06
mean     5.296568e+00
std      4.339355e+02
min      0.000000e+00
25%      1.030000e+00
50%      1.800000e+00
75%      3.540000e+00
max      3.303976e+05
Name: trip_distance, dtype: float64
La mediana de trip_distance es: 1.8


count    9.689116e+06
mean     1.976765e+01
std      2.047339e+01
min     -2.261200e+03
25%      9.300000e+00
50%      1.420000e+01
75%      2.315000e+01
max      2.261200e+03
Name: fare_amount, dtype: float64
La mediana de fare_amount es: 14.2


count    9.689116e+06
mean     1.371863e+00
std      1.826801e+00
min     -7.500000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      2.500000e+00
max      1.619000e+01
Name: extra, dtype: float64
La mediana de extra es: 1.0


count    9.6891